## HCBB gryphons_goal_3

In [1]:
import pandas as pd

## 1. What procedures could be reasonably associated with each individual Provider Type? There are 88 types of Providers.
- Filters for Physician/Other Supplier
- Entity Type = I
- Place of Service = F and/or O
- What is the normal ‘basket’ of procedures for each Provider Type?

## 2. Which procedure codes drive the most revenue for each Professional Provider Types?
- Exclude office visits

## Stretch: Create a process that predicts if an individual professional should be Provider Type ‘Orthopedic Surgery’ based on the procedure codes they have data for.
- How often does this data-driven expected Provider Type of ‘Orthopedic Surgery’ differ from the listed Provider Type for given professional?

In [2]:
providers_and_procedures = pd.read_pickle("../data/hcpcs_providers_clean.pkl")

In [3]:
providers_and_procedures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9416125 entries, 0 to 9416124
Data columns (total 23 columns):
National Provider Identifier                                int64
Last Name/Organization Name of the Provider                 object
First Name of the Provider                                  object
Credentials of the Provider                                 object
Entity Type of the Provider                                 object
Street Address 1 of the Provider                            object
City of the Provider                                        object
Zip Code of the Provider                                    object
State Code of the Provider                                  object
Country Code of the Provider                                object
Provider Type                                               object
Medicare Participation Indicator                            object
Place of Service                                            object
HCPCS Code       

In [4]:
providers_and_procedures.head()

,National Provider Identifier,Last Name/Organization Name of the Provider,First Name of the Provider,Credentials of the Provider,Entity Type of the Provider,Street Address 1 of the Provider,City of the Provider,Zip Code of the Provider,State Code of the Provider,Country Code of the Provider,...,HCPCS Code,HCPCS Description,HCPCS Drug Indicator,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Submitted Charge Amount,Average Medicare Payment Amount,Average Medicare Standardized Amount
0,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99217,Hospital observation care discharge,N,100.0,96,100,73.398800,325.780000,56.827200,57.492400
1,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99218,Hospital observation care typically 30 minutes,N,26.0,25,26,100.080000,449.000000,78.460000,79.306154
2,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99219,Hospital observation care typically 50 minutes,N,52.0,51,52,136.380000,614.000000,102.807692,103.895385
3,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99220,Hospital observation care typically 70 minutes...,N,59.0,59,59,190.363729,755.932203,141.293559,142.865763
4,1003000126,ENKESHAFI,ARDALAN,M.D.,I,900 SETON DR,CUMBERLAND,215021854,MD,US,...,99221,"Initial hospital inpatient care, typically 30 ...",N,16.0,16,16,101.680000,462.812500,79.710000,80.750000


In [5]:
providers_and_procedures['Place of Service'].value_counts()

O    5740315
F    3675810
Name: Place of Service, dtype: int64

In [7]:
providers_and_procedures['Provider Type'].value_counts()

Diagnostic Radiology                   1241400
Internal Medicine                      1118171
Family Practice                         969268
Nurse Practitioner                      560219
Cardiology                              445088
                                        ...   
Ambulance Service Provider                  42
Unknown Supplier/Provider Specialty         17
All Other Suppliers                         11
Medical Toxicology                           2
Slide Preparation Facility                   2
Name: Provider Type, Length: 88, dtype: int64

In [8]:
providers_and_procedures['HCPCS Description'].value_counts()

Established patient office or other outpatient visit, typically 15 minutes    457835
Established patient office or other outpatient, visit typically 25 minutes    428262
New patient office or other outpatient visit, typically 45 minutes            179569
New patient office or other outpatient visit, typically 30 minutes            178057
Subsequent hospital inpatient care, typically 25 minutes per day              175335
                                                                               ...  
Anesthesia for surgery of nose and sinus                                           1
Procalcitonin (hormone) level                                                      1
Clotting factor XIII (fibrin stabilizing) screening test                           1
Anesthesia for closed procedure on bones of forearm, wrist, or hand                1
Injection, progesterone, per 50 mg                                                 1
Name: HCPCS Description, Length: 5171, dtype: int64